In [1]:
import spacy
import timeit
import math
import pandas as pd
import matplotlib.pyplot as plt
from os import path
from collections import Counter
from lxml import etree
from glob import glob
from unicodedata import normalize

/home/humanum/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
%matplotlib inline

In [3]:
nlp = spacy.load('fr_core_news_lg')

In [36]:
#path_name = 'corpus_test/*.xml'
path_name = 'corpus_main/*.xml'
window = 1000
nombre_bigrammes = 100

In [5]:
def est_canon(tree):
    """fonction qui prend en parametre un arbre xml et qui retourne sous la forme d'un booleen si le document est classé comme canon ou non"""
    if tree.find(".//profileDesc") is not None:
        profil = tree.find(".//profileDesc")
        if profil.get("tag") == "canon":
            return True
        else:
            return False

In [6]:
def pipeline_spacy(path):
    pos_ko = ["NUM", "X", "SYM", "PUNCT", "SPACE"]
    list_lemma = []
    list_pos = []
    nombre_tokens = 0
    with open(path, encoding="utf8") as file:
        #on récupère l'arbre du document xml
        tree = etree.parse(file)
        #on récupère l'indice de canonicité du roman
        tag = est_canon(tree)
        if tag == True:
            print("canon")
        else:
            print("non_canon")
        #on récupère les paragraphes avec un xpath
        if tree.findall(".//p"):
            for paragraphe in tree.findall(".//p"):
                if paragraphe.text:
                    #on nettoie le texte et on le met dans la pipeline de spacy
                    clean_text = normalize("NFKD", paragraphe.text).lower()
                    docs = nlp(clean_text)
                    nombre_tokens += len(docs)
                    for token in docs:
                        #si le token est bien un mot on récupère son lemme
                        if token.pos_ not in pos_ko:
                            list_lemma.append(token.lemma_)
                            list_pos.append(token.pos_)
                            
    return list_lemma, list_pos, nombre_tokens, tag

In [7]:
def bigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de bi-grammes"""
    list_bigram = []
    for indice_token in range(len(list_token)-1):
        bigram = list_token[indice_token]+'_'+list_token[indice_token+1]
        list_bigram.append(bigram)
    return list_bigram

In [8]:
def trigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de tri-grammes"""
    list_trigram = []
    for indice_token in range(len(list_token)-2):
        trigram = list_token[indice_token]+'_'+list_token[indice_token+1]+'_'+list_token[indice_token+2]
        list_trigram.append(trigram)
    return list_trigram

In [9]:
def rollingntokens(list_tokens, n):
    """fonction qui prend en parametre une liste de tokens et un nombre n et decoupe cette liste en fragments de n tokens puis retourne une liste de listes de tokens"""
    i = 0 # i stocke l'indice auquel on est dans le rolling
    list_rolling = []
    while i-n < len(list_tokens):
        list_rolling.append(list_tokens[i:i+n])
        i+=n
    return list_rolling 

In [10]:
def rolling_operationnalisation(rolling_list, window, doc_name, nombre_bigrammes, feature):
    """fonction qui prend en parametre une liste de liste de token  et une fenetre et calcul le ratio type-token pour chaque liste et retourne une liste de liste de valeurs"""
    i = 1
    list_type_token = []
    list_entropy = []
    for list_tokens in rolling_list:
        if type(list_tokens) == list:
            table_freq = Counter(list_tokens)
            list_type_token.append(type_token(table_freq, window))
            list_entropy.append(entropy(table_freq, window, nombre_bigrammes))
            #On stocke chaque table de frequences pour les chunks de 1000 mots dans des dataframmes
            zipped = list(zip(table_freq.keys(), table_freq.values()))
            df = pd.DataFrame(zipped, columns=[feature, 'Nombre d\'apparitions'])
            df['Frequence d\'apparition'] = round(df['Nombre d\'apparitions']/sum(table_freq.values()),5)
            df.to_csv(r'tables_frequences/'+feature+'/'+doc_name+'_'+feature+str(i)+'freq.csv', index = False)
            i+=1
        
    return list_type_token, list_entropy

In [11]:
"""fonction qui prend en parametre une table de frequence et retourne le calcul du ratio type-token"""
def type_token(table_freq, window):
    #test de verification sur les valeurs recuperees
    if sum(table_freq.values()) == window:
        #On calcule le type_token ratio et on le retourne
        return round(len(table_freq)/sum(table_freq.values()),5)

In [12]:
def entropy(table_freq, window, nombre_bigrammes):
    """fonction qui prend en parametre une table de frequence et un nombre de bigramme et retourne leur entropie"""
    # initialisation de l'indice de shannon
    shannon_sum = 0
    #test de verification sur les valeurs recuperees
    if sum(table_freq.values()) == window:
        #On calcule l'entropie et on la retourne
        table_freq_select = dict(table_freq.most_common(nombre_bigrammes))
        for bigram in table_freq_select.keys():
            prop = table_freq_select[bigram]/window
            shannon_sum += prop * (math.log(prop, 5))   
        return round(shannon_sum * -1,2)

In [37]:
def moulinette(path_name, window, nombre_bigrammes):
    """fonction main qui utilise les fonctions précédentes et tourne sur le corpus"""
    canon = False
    i = 1
    nombre_total_tokens = 0
    
    annee_canon = []
    annee_archive = []
    
    
    #################### DATAFRAMES DES RESULTATS ###################

    ######## TT LEMMA ##########
    tt_canon_lemma = pd.DataFrame()
    tt_archive_lemma = pd.DataFrame()
        
    tt_canon_bigram_lemma = pd.DataFrame()
    tt_archive_bigram_lemma = pd.DataFrame()
        
    tt_canon_trigram_lemma = pd.DataFrame()
    tt_archive_trigram_lemma = pd.DataFrame()
        
    ######## TT POS ########## 
    tt_canon_pos = pd.DataFrame()
    tt_archive_pos = pd.DataFrame()
        
    tt_canon_bigram_pos = pd.DataFrame()
    tt_archive_bigram_pos = pd.DataFrame()
        
    tt_canon_trigram_pos = pd.DataFrame()
    tt_archive_trigram_pos = pd.DataFrame()
        
    ######## ENTROPY LEMMA ########## 

    entropy_canon_lemma = pd.DataFrame()
    entropy_archive_lemma = pd.DataFrame()
        
    entropy_canon_bigram_lemma = pd.DataFrame()
    entropy_archive_bigram_lemma = pd.DataFrame()
        
    entropy_canon_trigram_lemma = pd.DataFrame()
    entropy_archive_trigram_lemma = pd.DataFrame()
        
    ######## ENTROPY POS ########## 
        
    entropy_canon_pos = pd.DataFrame()
    entropy_archive_pos = pd.DataFrame()
        
    entropy_canon_bigram_pos = pd.DataFrame()
    entropy_archive_bigram_pos = pd.DataFrame()
        
    entropy_canon_trigram_pos = pd.DataFrame()
    entropy_archive_trigram_pos = pd.DataFrame()
        

    
    print("\n\nBEGIN PROCESSING CORPUS-----------")
    
    for doc in glob(path_name):
        
        print("\n\nBEGIN PROCESSING NOVEL-----------")

        
        doc_name = path.splitext(path.basename(doc))[0]
        date = doc_name.split("_")[0]
        print(doc_name)
        
        #On recupere le texte des romans sous forme de listes de lemmes et de pos grâce à spacy
        
        list_lemma, list_pos, nombre_tokens, canon = pipeline_spacy(doc)
        print("PIPELINE SPACY ----------- OK")
        
        print("NOMBRE TOKENS = ", nombre_tokens)
        
        nombre_total_tokens += nombre_tokens
        
        rolling_list_lemma = rollingntokens(list_lemma, window)
        rolling_list_bigram_lemma = rollingntokens(bigrammize(list_lemma), window)
        rolling_list_trigram_lemma = rollingntokens(trigrammize(list_lemma), window)
        
        rolling_list_pos = rollingntokens(list_pos, window)
        rolling_list_bigram_pos = rollingntokens(bigrammize(list_pos), window)
        rolling_list_trigram_pos = rollingntokens(trigrammize(list_pos), window)
        print("ROLLING ----------- OK")
        
        #on effectue nos calculs type token et entropie sur ces listes 
        
        ########## LEMMES ###########
        list_type_token_lemma, list_entropy_lemma = rolling_operationnalisation(rolling_list_lemma, window, doc_name, nombre_bigrammes, 'lemma')
        list_type_token_bigram_lemma, list_entropy_bigram_lemma = rolling_operationnalisation(rolling_list_bigram_lemma, window, doc_name, nombre_bigrammes, 'bigram_lemma')
        list_type_token_trigram_lemma, list_entropy_trigram_lemma = rolling_operationnalisation(rolling_list_trigram_lemma, window, doc_name, nombre_bigrammes, 'trigram_lemma')
        print("OPERATIONS LEMMAS ----------- OK")
        
        ########### POS ###########
        list_type_token_pos, list_entropy_pos = rolling_operationnalisation(rolling_list_pos, window, doc_name, nombre_bigrammes, 'pos')
        list_type_token_bigram_pos, list_entropy_bigram_pos = rolling_operationnalisation(rolling_list_bigram_pos, window, doc_name, nombre_bigrammes, 'bigram_pos')
        list_type_token_trigram_pos, list_entropy_trigram_pos = rolling_operationnalisation(rolling_list_trigram_pos, window, doc_name, nombre_bigrammes, 'trigram_pos')
        print("OPERATIONS POS ----------- OK")
        
        
        print("BEGIN SAVING RESULTS -----------")
        
                
        if canon:

            temp_canon = pd.Series(list_type_token_lemma, name=doc_name, dtype = 'float64')
            tt_canon_lemma = pd.concat([tt_canon_lemma, temp_canon], axis=1)
            
            temp_canon = pd.Series(list_type_token_bigram_lemma, name=doc_name, dtype = 'float64')
            tt_canon_bigram_lemma = pd.concat([tt_canon_bigram_lemma, temp_canon], axis=1)
            
            temp_canon = pd.Series(list_type_token_trigram_lemma, name=doc_name, dtype = 'float64')
            tt_canon_trigram_lemma = pd.concat([tt_canon_trigram_lemma, temp_canon], axis=1)
            
            
            temp_canon = pd.Series(list_type_token_pos, name=doc_name, dtype = 'float64')
            tt_canon_pos = pd.concat([tt_canon_pos, temp_canon], axis=1)
            
            temp_canon = pd.Series(list_type_token_bigram_pos, name=doc_name, dtype = 'float64')
            tt_canon_bigram_pos = pd.concat([tt_canon_bigram_pos, temp_canon], axis=1)
            
            temp_canon = pd.Series(list_type_token_trigram_pos, name=doc_name, dtype = 'float64')
            tt_canon_trigram_pos = pd.concat([tt_canon_trigram_pos, temp_canon], axis=1)
            
            
            temp_canon = pd.Series(list_entropy_lemma, name=doc_name, dtype = 'float64')
            entropy_canon_lemma = pd.concat([entropy_canon_lemma, temp_canon], axis=1)
            
            temp_canon = pd.Series(list_entropy_bigram_lemma, name=doc_name, dtype = 'float64')
            entropy_canon_bigram_lemma = pd.concat([entropy_canon_bigram_lemma, temp_canon], axis=1)
            
            temp_canon = pd.Series(list_entropy_trigram_lemma, name=doc_name, dtype = 'float64')
            entropy_canon_trigram_lemma = pd.concat([entropy_canon_trigram_lemma, temp_canon], axis=1)
            
            
            temp_canon = pd.Series(list_entropy_pos, name=doc_name, dtype = 'float64')
            entropy_canon_pos = pd.concat([entropy_canon_pos, temp_canon], axis=1)
            
            temp_canon = pd.Series(list_entropy_bigram_pos, name=doc_name, dtype = 'float64')
            entropy_canon_bigram_pos = pd.concat([entropy_canon_bigram_pos, temp_canon], axis=1)
            
            temp_canon = pd.Series(list_entropy_trigram_pos, name=doc_name, dtype = 'float64')
            entropy_canon_trigram_pos = pd.concat([entropy_canon_trigram_pos, temp_canon], axis=1)
            
            
            annee_canon.append(date)
            
        else:
            
            temp_archive = pd.Series(list_type_token_lemma, name=doc_name, dtype = 'float64')
            tt_archive_lemma = pd.concat([tt_archive_lemma, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_type_token_bigram_lemma, name=doc_name, dtype = 'float64')
            tt_archive_bigram_lemma = pd.concat([tt_archive_bigram_lemma, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_type_token_trigram_lemma, name=doc_name, dtype = 'float64')
            tt_archive_trigram_lemma = pd.concat([tt_archive_trigram_lemma, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_type_token_pos, name=doc_name, dtype = 'float64')
            tt_archive_pos = pd.concat([tt_archive_pos, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_type_token_bigram_pos, name=doc_name, dtype = 'float64')
            tt_archive_bigram_pos = pd.concat([tt_archive_bigram_pos, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_type_token_trigram_pos, name=doc_name, dtype = 'float64')
            tt_archive_trigram_pos = pd.concat([tt_archive_trigram_pos, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_entropy_lemma, name=doc_name, dtype = 'float64')
            entropy_archive_lemma = pd.concat([entropy_archive_lemma, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_entropy_bigram_lemma, name=doc_name, dtype = 'float64')
            entropy_archive_bigram_lemma = pd.concat([entropy_archive_bigram_lemma, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_entropy_trigram_lemma, name=doc_name, dtype = 'float64')
            entropy_archive_trigram_lemma = pd.concat([entropy_archive_trigram_lemma, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_entropy_pos, name=doc_name, dtype = 'float64')
            entropy_archive_pos = pd.concat([entropy_archive_pos, temp_archive], axis=1)
            
            
            temp_archive = pd.Series(list_entropy_bigram_pos, name=doc_name, dtype = 'float64')
            entropy_archive_bigram_pos = pd.concat([entropy_archive_bigram_pos, temp_archive], axis=1)
            
            temp_archive = pd.Series(list_entropy_trigram_pos, name=doc_name, dtype = 'float64')
            entropy_archive_trigram_pos = pd.concat([entropy_archive_trigram_pos, temp_archive], axis=1)
            
            annee_archive.append(date)

        i+=1

        print("END SAVING RESULTS -----------")
        print("END PROCESSING NOVEL --------------\n\n")
        print("PROGRESSION ", round(i/30,3),'% COMPLETED\n')
        
    print("\n\nNOMBRE TOTAL TOKENS = ", nombre_total_tokens)
    print("\n\nEND PROCESSING CORPUS --------------\n\n")
            
    return nombre_total_tokens, annee_canon, annee_archive, tt_canon_lemma, tt_archive_lemma, tt_canon_bigram_lemma, tt_archive_bigram_lemma, tt_canon_trigram_lemma, tt_archive_trigram_lemma, tt_canon_pos, tt_archive_pos, tt_canon_bigram_pos, tt_archive_bigram_pos, tt_canon_trigram_pos, tt_archive_trigram_pos, entropy_canon_lemma, entropy_archive_lemma, entropy_canon_bigram_lemma, entropy_archive_bigram_lemma, entropy_canon_trigram_lemma, entropy_archive_trigram_lemma, entropy_canon_pos, entropy_archive_pos, entropy_canon_bigram_pos, entropy_archive_bigram_pos, entropy_canon_trigram_pos, entropy_archive_trigram_pos

In [38]:
starttime = timeit.default_timer()
nombre_total_tokens, annee_canon, annee_archive, tt_canon_lemma, tt_archive_lemma, tt_canon_bigram_lemma, tt_archive_bigram_lemma, tt_canon_trigram_lemma, tt_archive_trigram_lemma, tt_canon_pos, tt_archive_pos, tt_canon_bigram_pos, tt_archive_bigram_pos, tt_canon_trigram_pos, tt_archive_trigram_pos, entropy_canon_lemma, entropy_archive_lemma, entropy_canon_bigram_lemma, entropy_archive_bigram_lemma, entropy_canon_trigram_lemma, entropy_archive_trigram_lemma, entropy_canon_pos, entropy_archive_pos, entropy_canon_bigram_pos, entropy_archive_bigram_pos, entropy_canon_trigram_pos, entropy_archive_trigram_pos = moulinette(path_name, window, nombre_bigrammes) 
print("Le temps total d'execution en secondes est de : ", timeit.default_timer() - starttime)



BEGIN PROCESSING CORPUS-----------


BEGIN PROCESSING NOVEL-----------
1880_Ponson-du-Terrail-Pierre-Alexis-de_Les-Aventures-du-capitaine-La-Palisse
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  94451
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  0.067 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1939_Fargue-Leon-Paul_Le-Pieton-De-Paris
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  80917
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  0.1 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1821_Arlincourt-Charles-Victor_Le-Solitaire
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  79868
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS PO

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  33763
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  0.833 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1904_Moselly-Emile_Jean-des-Brebis-ou-Le-livre-de-la-misere
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  56474
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  0.867 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1956_Delly_Annonciade
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  60260
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  0.9 % COMPLETED



BEGIN P

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  71809
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  1.6 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1968_Gary-Romain_La-Tete-coupable
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  109165
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  1.633 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1846_Musset-Paul-de_Monsieur-le-Vent-et-Madame-la-Pluie
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  21824
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  1.667 % COMPLETED



BE

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  29358
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  2.367 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1887_Villiers-de-l'Isle-Adam_Tribulat-Bonhomet
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  40798
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  2.4 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1845_Balzac-Honore-de_HonorineFC
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  35579
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  2.433 % COMPLETED



BEGIN PROCESSING 

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  40767
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  3.133 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1878_Sauniere-Paul_Le-Capitaine-Belle-Humeur
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  90833
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  3.167 % COMPLETED



BEGIN PROCESSING NOVEL-----------
2012_Jourde-Pierre_Le-Marechal-absolu
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  301704
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  3.2 % COMPLETED



BEGIN

...

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  51108
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  95.633 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1911_Montesquiou-Fezensac-Robert-de_La-petite-mademoiselle
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  48429
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  95.667 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1927_Crevel-René_Babylone
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  46904
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  95.7 % COMPLETED



BEGIN

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  64297
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  96.4 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1986_Quignard-Pascal_Le-salon-du-Wurtemberg
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  138030
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  96.433 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1891_Barres-Maurice_Le-culte-du-moi-3-Le-jardin-de-Berenice
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  38611
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  96.467 % CO

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  55660
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  97.167 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1876_Sand-George_Contes-d-une-grand-mere
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  136457
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  97.2 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1904_Noailles-Anna-de_Le-visage-emerveille
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  27752
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  97.233 % COMPLETED



BEGIN PRO

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  125010
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  97.933 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1889_Verne-Jules_Au-XXIXe-siecle-ou-la-journee-d-un-journaliste-americain
canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  7015
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  97.967 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1982_Chessex-Jacques_Judas-le-transparent
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  55953
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRES

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  49216
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  98.7 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1860_Privat-d-Anglemont-Alexandre_Paris-anecdote
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  76858
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  98.733 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1871_Erckmann-Chatrian_Les-deux-freres
non_canon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  79821
ROLLING ----------- OK
OPERATIONS LEMMAS ----------- OK
OPERATIONS POS ----------- OK
BEGIN SAVING RESULTS -----------
END SAVING RESULTS -----------
END PROCESSING NOVEL --------------


PROGRESSION  98.767 % COMPLETED



In [39]:
tt_canon_lemma.to_csv(r'moulinette_results/tt_canon_lemma.csv', index = False, header=True)
tt_archive_lemma.to_csv(r'moulinette_results/tt_archive_lemma.csv', index = False, header=True)
tt_canon_bigram_lemma.to_csv(r'moulinette_results/tt_canon_bigram_lemma.csv', index = False, header=True)
tt_archive_bigram_lemma.to_csv(r'moulinette_results/tt_archive_bigram_lemma.csv', index = False, header=True)
tt_canon_trigram_lemma.to_csv(r'moulinette_results/tt_canon_trigram_lemma.csv', index = False, header=True)
tt_archive_trigram_lemma.to_csv(r'moulinette_results/tt_archive_trigram_lemma.csv', index = False, header=True)

tt_canon_pos.to_csv(r'moulinette_results/tt_canon_pos.csv', index = False, header=True)
tt_archive_pos.to_csv(r'moulinette_results/tt_archive_pos.csv', index = False, header=True)
tt_canon_bigram_pos.to_csv(r'moulinette_results/tt_canon_bigram_pos.csv', index = False, header=True)
tt_archive_bigram_pos.to_csv(r'moulinette_results/tt_archive_bigram_pos.csv', index = False, header=True)
tt_canon_trigram_pos.to_csv(r'moulinette_results/tt_canon_trigram_pos.csv', index = False, header=True)
tt_archive_trigram_pos.to_csv(r'moulinette_results/tt_archive_trigram_pos.csv', index = False, header=True)

entropy_canon_lemma.to_csv(r'moulinette_results/entropy_canon_lemma.csv', index = False, header=True)
entropy_archive_lemma.to_csv(r'moulinette_results/entropy_archive_lemma.csv', index = False, header=True)
entropy_canon_bigram_lemma.to_csv(r'moulinette_results/entropy_canon_bigram_lemma.csv', index = False, header=True)
entropy_archive_bigram_lemma.to_csv(r'moulinette_results/entropy_archive_bigram_lemma.csv', index = False, header=True)
entropy_canon_trigram_lemma.to_csv(r'moulinette_results/entropy_canon_trigram_lemma.csv', index = False, header=True)
entropy_archive_trigram_lemma.to_csv(r'moulinette_results/entropy_archive_trigram_lemma.csv', index = False, header=True)

entropy_canon_pos.to_csv(r'moulinette_results/entropy_canon_pos.csv', index = False, header=True)
entropy_archive_pos.to_csv(r'moulinette_results/entropy_archive_pos.csv', index = False, header=True)
entropy_canon_bigram_pos.to_csv(r'moulinette_results/entropy_canon_bigram_pos.csv', index = False, header=True)
entropy_archive_bigram_pos.to_csv(r'moulinette_results/entropy_archive_bigram_pos.csv', index = False, header=True)
entropy_canon_trigram_pos.to_csv(r'moulinette_results/entropy_canon_trigram_pos.csv', index = False, header=True)
entropy_archive_trigram_pos.to_csv(r'moulinette_results/entropy_archive_trigram_pos.csv', index = False, header=True)


In [ ]:
#Le temps total d'execution en secondes est de :  49448.30465074401
#NOMBRE TOTAL TOKENS =  242051348

In [32]:
print("_____________ tt_canon_lemma _____________\n", tt_canon_lemma)
print("_____________ tt_archive_lemma _____________\n" ,tt_archive_lemma)
print("_____________ tt_canon_bigram_lemma _____________\n",tt_canon_bigram_lemma)
print("_____________ tt_archive_bigram_lemma _____________\n",tt_archive_bigram_lemma)
print("_____________ tt_canon_trigram_lemma _____________\n",tt_canon_trigram_lemma)
print("_____________ tt_archive_trigram_lemma_____________\n",tt_archive_trigram_lemma)
print("_____________ tt_canon_pos _____________\n",tt_canon_pos)
print("_____________ tt_archive_pos _____________\n",tt_archive_pos)
print("_____________ tt_canon_bigram_pos_____________\n",tt_canon_bigram_pos)
print("_____________ tt_archive_bigram_pos _____________\n",tt_archive_bigram_pos)
print("_____________ tt_canon_trigram_pos _____________\n",tt_canon_trigram_pos)
print("_____________ tt_archive_trigram_pos _____________\n",tt_archive_trigram_pos)


print("_____________ entropy_canon_lemma _____________\n",entropy_canon_lemma)
print("_____________ entropy_archive_lemma _____________\n",entropy_archive_lemma)
print("_____________ entropy_canon_bigram_lemma _____________\n",entropy_canon_bigram_lemma)
print("_____________ entropy_archive_bigram_lemma _____________\n",entropy_archive_bigram_lemma)
print("_____________ entropy_canon_trigram_lemma _____________\n",entropy_canon_trigram_lemma)
print("_____________ entropy_archive_trigram_lemma\n",entropy_archive_trigram_lemma)
print("_____________ entropy_canon_pos _____________\n",entropy_canon_pos)
print("_____________ entropy_archive_pos _____________\n",entropy_archive_pos)
print("_____________ entropy_canon_bigram_pos\n",entropy_canon_bigram_pos)
print("_____________ entropy_archive_bigram_pos _____________\n",entropy_archive_bigram_pos)
print("_____________ entropy_canon_trigram_pos _____________\n",entropy_canon_trigram_pos)
print("_____________ entropy_archive_trigram_pos _____________\n",entropy_archive_trigram_pos)


_____________ tt_canon_lemma _____________
      1822_Hugo-Victor_Han-d-Islande  1830_Stendhal_Le-Rouge-et-le-noir  \
0                             0.418                              0.401   
1                             0.434                              0.412   
2                             0.455                              0.372   
3                             0.423                              0.367   
4                             0.459                              0.378   
..                              ...                                ...   
179                             NaN                              0.417   
180                             NaN                              0.379   
181                             NaN                              0.408   
182                             NaN                                NaN   
183                             NaN                                NaN   

     1823_Duras-Claire-de-Durfort_Ourika  
0                       

In [ ]:
#tt_canon_lemma[doc_name] = pd.Series(list_type_token_lemma)
#df = pd.DataFrame({'text': [], 'lemma': [], 'pos': [], 'is_alpha': [],'is_stop': []})
#df.to_csv(r'C:\Users\jeanb\Documents\memoire\en_cours\test.csv', index = False)